In [8]:
import socket

afqdn = socket.getfqdn() # your /etc/hosts should be set up to contain fully qualified domain name of the machine
afqdn

'nnlicv431.inn.intel.com'

## 1. Make sure you can run the `fx` command.

## 2. Create a workspace for the new federation project.

In [9]:
!fx workspace create --prefix ~/workspace --template torch_cnn_histology
%cd ~/workspace

Intel openfl - Secure Federated Learning at the Edge™                           

Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates
You are using pip version 18.1, however version 20.3.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

New workspace directory structure:
workspace
├── code
│   ├── keras_cnn.py
│   ├── __pycache__
│   │   ├── __init__.cpython-36.pyc
│   │   ├── pt_cnn.cpython-36.pyc
│   │   ├── histology_utils.cpython-36.pyc
│   │   └── pthistology_inmemory.cpython-36.pyc
│   ├── mnist_utils.py
│   ├── tfmnist_inmemory.py
│   ├── pthistology_inmemory.py
│   ├── __init__.py
│   ├── histology_utils.py
│   ├── requirements.txt
│   └── pt_cnn.py
├── agg_to_col_two_signed_cert.zip
├── requirements.export.txt
├── cert
│   ├── ca
│   │   ├── root-ca.csr
│   │   ├── signing-ca
│   │   ├── root-ca.crt
│   │   ├── signing-ca.crt
│   │   ├── root-ca
│   │   └── signing-ca.csr
│   ├── client
│   │  

## 3. Initialize the plan
Although it is possible to train models from scratch, it is assumed that in many cases the federation may perform fine-tuning of a previously-trained model. For this reason, the pre-trained weights for the model will be stored within protobuf files on the aggregator and passed to the collaborators during initialization. As seen in the YAML file, the protobuf file with the initial weights is expected to be found in the file `torch_cnn_mnist_init.pbuf`. Let's just create an initial set of random model weights and put it into that file by running the command:

In [10]:
!fx plan initialize -a $afqdn

Intel openfl - Secure Federated Learning at the Edge™                           

[12:53:26] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1608544406.1747818-79159;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1608544406.1772122-758113;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFAULTS for section collaborator from file plan/defaults/collaborator.yaml.                                              ]8;id=1608544406.180026-473780;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFA

[12:53:28] INFO     Histology > X_train Shape : (2000, 3, 150, 150)                                                                       ]8;id=1608544408.8970194-914918;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:136
           INFO     Histology > y_train Shape : (2000,)                                                                                   ]8;id=1608544408.8990047-114452;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:137
           INFO     Histology > Train Samples : 2000                                                                                      ]8;id=1608544408.900808-396606;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:138
           INFO     Histology > Valid Samples : 500                                                                                       ]8;id=1608544408.9026072-350523;file:///home/itrushkin/workspace/code/histology_utils

## 4. Run the Certificate Authority command.
This will setup the Aggregator node as the Certificate Authority for the Federation. All certificates will be signed by the aggregator. The command will create a simple database file to keep track of all issued certificates.

In [11]:
!fx workspace certify

Intel openfl - Secure Federated Learning at the Edge™                           

Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request
1.4 Create CA Certificate
2.  Create Signing Certificate
2.1 Create Directories
2.2 Create Database
2.3 Create Signing Certificate CSR
2.4 Sign Signing Certificate CSR
3   Create Certificate Chain

Done.

 ✔️ OK


## 5. Run the aggregator certificate creation command.

In [12]:
!fx aggregator generate-cert-request --fqdn $afqdn

Intel openfl - Secure Federated Learning at the Edge™                           

Creating AGGREGATOR certificate key pair with following settings: CN=nnlicv431.inn.intel.com, SAN=DNS:nnlicv431.inn.intel.com
  Moving AGGREGATOR certificate key pair to: cert/server

 ✔️ OK


## 6. Run the aggregator certificate signing command

In [13]:
!fx aggregator certify -s --fqdn $afqdn

Intel openfl - Secure Federated Learning at the Edge™                           

The CSR Hash for file agg_nnlicv431.inn.intel.com.csr =  cdbb8d85db404f0313f5b7b0e2e3ae31820badd27d22128c75ec0759f1c62349

 Signing AGGREGATOR certificate key pair
  Moving AGGREGATOR certificate key pair to: cert/server

 ✔️ OK


## 7. Export the workspace so that it can be imported to the collaborator nodes.

In [14]:
!fx workspace export

Intel openfl - Secure Federated Learning at the Edge™                           

[12:54:24] INFO     FL-Plan hash is 7da62f28238a0d52ab0db363b2fe9a46                                                                                 ]8;id=1608544464.8426452-665709;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:204
           INFO     plan.yaml is already frozen                                                                                                       ]8;id=1608544464.8445084-719071;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:45
requirements.export.txt written.
Workspace exported to archive: workspace.zip

 ✔️ OK


## 14. Sign collaborators certificates

In [15]:
!fx collaborator certify -s --request-pkg col_one_to_agg_cert_request.zip

Intel openfl - Secure Federated Learning at the Edge™                           

The CSR Hash for file col_one.csr =  587c0d730e8126ab9243d110411c93e8975f1f93523867e9556d330c0f6946fd

 Signing COLLABORATOR certificate key pair
  Moving COLLABORATOR certificate key pair to: cert/client

Registering one in plan/cols.yaml

 ✔️ OK


In [16]:
!fx collaborator certify -s --request-pkg col_two_to_agg_cert_request.zip

Intel openfl - Secure Federated Learning at the Edge™                           

The CSR Hash for file col_two.csr =  1d9d444313e3c60fb9bc86749d89d3ba3990e759a921147dbd49d75b96e09d91

 Signing COLLABORATOR certificate key pair
  Moving COLLABORATOR certificate key pair to: cert/client

Registering two in plan/cols.yaml

 ✔️ OK


## Start the aggregator.

In [17]:
!fx aggregator start

Intel openfl - Secure Federated Learning at the Edge™                           

[12:55:21] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1608544521.2594137-868739;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1608544521.2617695-538518;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFAULTS for section collaborator from file plan/defaults/collaborator.yaml.                                              ]8;id=1608544521.2645357-814914;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DE

           INFO     Building 🡆 Object Aggregator from openfl.component Module.                                                                       ]8;id=1608544521.9139788-32234;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:165
           INFO     Settings 🡆 {'db_store_rounds': 1, 'best_state_path': 'save/torch_cnn_histology_best.pbuf', 'init_state_path':                    ]8;id=1608544521.9172027-448856;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:167
                    'save/torch_cnn_histology_init.pbuf', 'last_state_path': 'save/torch_cnn_histology_last.pbuf', 'rounds_to_train': 20,                       
                    'aggregator_uuid': 'aggregator_plan.yaml_7da62f28', 'federation_uuid': 'plan.yaml_7da62f28', 'authorized_cols': ['one', 'two'],             
                    'assigner': <openfl.component.assigner.random_grouped_assigner.Rand

[12:56:10] INFO     Saving round 1 model...                                                                                                    ]8;id=1608544570.8597317-261254;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:880
[12:56:11] INFO     Starting round 1...                                                                                                        ]8;id=1608544571.1208427-804820;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:888
           INFO     Aggregator GetTasks function reached from collaborator two...                                                              ]8;id=1608544571.1582103-45935;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:289
           INFO     Sending tasks to collaborator two for round 1            

[12:56:43] INFO     Collaborator one is sending task results for aggregated_model_validation, round 2                                          ]8;id=1608544603.7762597-801704;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:505
[12:56:46] INFO     Aggregator GetTasks function reached from collaborator two...                                                              ]8;id=1608544606.7250652-491275;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:289
           INFO     Sending tasks to collaborator two for round 2                                                                              ]8;id=1608544606.7309747-275964;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:333
[12:56:49] INFO     Collaborator two is sending task results for aggregated_

[12:57:21] INFO     Collaborator one is sending task results for aggregated_model_validation, round 3                                          ]8;id=1608544641.264928-565254;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:505
[12:57:27] INFO     Collaborator two is sending task results for train, round 3                                                                ]8;id=1608544647.9396842-972745;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:505
[12:57:33] INFO     Collaborator two is sending task results for locally_tuned_model_validation, round 3                                       ]8;id=1608544653.164197-715285;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:505
           INFO     Aggregator GetTasks function reached from collaborator two

...

[13:03:36] INFO     Collaborator two is sending task results for train, round 15                                                               ]8;id=1608545016.9579902-246729;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:505
[13:03:37] INFO     Collaborator one is sending task results for aggregated_model_validation, round 15                                         ]8;id=1608545017.4535534-247591;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:505
           INFO     Collaborator two is sending task results for locally_tuned_model_validation, round 15                                      ]8;id=1608545017.778868-269244;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:505
           INFO     Aggregator GetTasks function reached from collaborator tw

[13:04:22] INFO     Collaborator one is sending task results for locally_tuned_model_validation, round 16                                      ]8;id=1608545062.3404496-751668;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:505
           INFO     Aggregator GetTasks function reached from collaborator one...                                                              ]8;id=1608545062.3652058-648612;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:289
           INFO     Collaborator two is sending task results for locally_tuned_model_validation, round 16                                      ]8;id=1608545062.584338-312812;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:505
           INFO     aggregated_model_validation task metrics...              

           INFO     acc:    0.7738                                                                                                             ]8;id=1608545091.308462-35411;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:751
           INFO     locally_tuned_model_validation task metrics...                                                                             ]8;id=1608545091.3115044-738157;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:683
           INFO     acc:    0.7978                                                                                                             ]8;id=1608545091.3236618-429787;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:751
           INFO     train task metrics...                                     

           INFO     CrossEntropyLoss:       0.6044                                                                                             ]8;id=1608545122.9804955-524993;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:751
[13:05:25] INFO     Saving round 19 model...                                                                                                   ]8;id=1608545125.4893713-602401;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:880
           INFO     Starting round 19...                                                                                                       ]8;id=1608545125.7714782-29932;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:888
           INFO     Aggregator GetTasks function reached from collaborator tw

[13:05:58] INFO     Aggregator GetTasks function reached from collaborator two...                                                              ]8;id=1608545158.2902896-379830;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:289
           INFO     Sending signal to collaborator two to shutdown...                                                                          ]8;id=1608545158.2967734-690187;file:///home/itrushkin/venvs/openfl/lib/python3.6/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:294

 ✔️ OK


At this point, the aggregator is running and waiting for the collaborators to connect. When all of the collaborators connect, the aggregator starts training. When the last round of training is complete, the aggregator stores the final weights